## Ada boost classifier


In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

In [2]:
dataset = pd.read_csv('../Data/imputedWQ.csv', parse_dates=True, header=0, index_col=0)

In [3]:
testset = pd.read_csv('../Data/testing2018-mean_imputed.csv', parse_dates=True, header=0, index_col=0)

In [7]:
X = dataset.drop('EVENT', axis=1)  
y = dataset['EVENT']  

In [8]:
X.drop(['Leit','Cl'], axis=1,inplace=True)
X.head(2)

,Tp,pH,Redox,Trueb,Cl_2,Fm,Fm_2
Time,,,,,,,
2016-08-03 09:49:00,6.5,8.36,749.0,11.0,118.0,1677.0,695.0
2016-08-03 09:50:00,6.5,8.36,749.0,11.0,118.0,1561.0,696.0


# TimeSeriesSplit
## Special case of KFold
### Param: n_splits according to the number of months

P.s: even though this probably would not result in accurate month separation, probably would go more with accurate samples separation

In [9]:
tss = TimeSeriesSplit(n_splits=4)
# kn = KNeighborsClassifier(n_neighbors=5, algorithm='brute') 
# sc = cross_val_score(kn, X, y, cv=tss, scoring='accuracy')

In [15]:
dataset.groupby(dataset.index.month).sum()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
8,290766.552632,6801.383362,343517.849450,3.096140e+07,8.664862e+06,787160.949453,4.009652e+06,6.738802e+07,3.886867e+07,165.0
9,353961.000000,7172.786251,361367.891324,3.252038e+07,9.111793e+06,865092.986451,4.396304e+06,6.858502e+07,4.055888e+07,1310.0
10,431439.430435,7384.794348,374076.693977,3.364090e+07,9.410047e+06,710263.140783,4.398462e+06,6.299596e+07,4.047184e+07,174.0
11,112340.400000,1736.240000,88673.130000,7.957033e+06,2.056404e+06,144211.260000,9.913454e+05,1.533976e+07,9.644754e+06,77.0


In [16]:
testset.groupby(testset.index.month).sum()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
1,168946.9,6486.070000,367918.485157,3.433636e+07,1.030073e+07,810.50142,4804.052413,65217445.0,45054296.0,773.0
2,65273.9,2560.330000,144423.580000,1.363104e+07,3.474466e+06,286.97000,1847.039119,26277783.0,18979849.0,895.0
11,183728.9,3401.380063,177387.328683,1.620504e+07,4.328124e+06,425.34816,2341.372000,30974768.0,19343072.0,352.0
12,171830.7,4713.325094,260945.894188,2.417727e+07,7.182003e+06,634.66800,3317.529000,44719245.0,26572835.0,309.0


## TimeSeries CV

- Applying TimeSeriesSplit in cross validation with multiple scores
    - F1-macro (alter. f1_weighted, f1_micro)
    - Precision
    - Recall
    
P.s errors result because the classifier on specific folds couldn't identify at all one class and probably gave just true or false ( I guess false in this case because of high class-imbalance)

In [10]:
scoring = ['f1','f1_micro', 'f1_macro','precision','recall']

In [11]:
seed=7
num_trees = 30
model = AdaBoostClassifier(DecisionTreeClassifier(max_depth=4), algorithm="SAMME", n_estimators=num_trees, random_state=seed)


In [27]:
#print("Model(RF, 2): ({0}, {1}, {2}, {3},{4})\n".format(model['test_f1'].mean(),model['test_f1_micro'].mean() , model['test_f1_macro'].mean(), model['test_precision'].mean(), model['test_recall'].mean()))

In [12]:
model.fit(X, y)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=30, random_state=7)

In [13]:
y_pred = model.predict(X)

In [14]:
f1_score(y_pred, y)
#f1_score(y_pred, y, average='macro')

0.9909647333139027

## GridSearchCV
### Hyperparameter optimizations for Adaboost classifier

- Scoring F1_macro can be changed to precision, recall or f1_weighted or f1_micro and possibly you would get different parameters.

In [16]:
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = {'n_estimators': np.arange(10,300,10),
              'base_estimator': [DecisionTreeClassifier(max_depth=3), DecisionTreeClassifier(max_depth=4), DecisionTreeClassifier(max_depth=5)]}
grid = GridSearchCV(AdaBoostClassifier(), param_grid, cv=tss, scoring="f1", n_jobs=-1)

In [17]:
grid.fit(X,y)

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/li

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/li

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=4),
       error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'n_estimators': array([ 10,  20,  30,  40,  50,  60,  70,  80,  90, 100, 110, 120, 130,
       140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260,
       270, 280, 290]), 'base_estimator': [DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            ...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [18]:
grid.best_params_

{'base_estimator': DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False, random_state=None,
             splitter='best'), 'n_estimators': 30}

In [19]:
model = grid.best_estimator_

In [20]:
results = cross_validate(model, X, y, cv=tss, scoring=scoring, n_jobs=-1)

In [21]:
print("Model(adab, 3, 30): ({0}, {1}, {2})\n".format(results['test_f1_macro'].mean(),results['test_f1'].mean(), results['test_f1_micro'].mean()))

Model(adab, 3, 30): (0.8043512506055397, 0.6141283922542657, 0.9893597965105865)



## Testing on Test set(unseen)

In [22]:
X_test = testset.drop('EVENT', axis=1)  
y_test = testset['EVENT']  

In [23]:
X_test.drop(['Leit','Cl'], axis=1,inplace=True)
X_test.head(2)

,Tp,pH,Redox,Trueb,Cl_2,Fm,Fm_2
Time,,,,,,,
2016-11-08 07:55:00,10.1,8.41,762.0,0.022,0.106,1818.0,920.0
2016-11-08 07:56:00,10.1,8.41,762.0,0.022,0.106,1805.0,927.0


In [24]:
predicted = model.predict(X_test)
predicted

array([False, False, False, ..., False, False, False])

In [25]:
confusion_matrix(y_test, predicted)

array([[115150,    487],
       [  1543,    786]])

In [26]:
print(f1_score(y_test,predicted))
print(recall_score(y_test,predicted))
print(precision_score(y_test,predicted))

0.43642420877290394
0.3374838986689566
0.617439120188531
